In [1]:
import pandas
import re
import math
dRaw = pandas.read_csv("./KA_DATA/DEMO_OUTPUT-Table 1.csv")

outputCSVHeaders = ['AGE', 'ALGOTYPE', 'DEMO', 'OVERALL.RESPONSE', 'DOB', 'GENDER', 'ID', 'OVERALL.RESPONSE.NO', 'RESPONSE.NO', 'ACTION.NO', 'BUCKET', 'RESPONSE']

In [2]:
dRaw.head()

,ID,DOB,AGE,GENDER,SORT_DEMO,SORT_OUTPUT,DOUBLE_DEMO,DOUBLE_OUTPUT,HITCH_DEMO,HITCH_OUTPUT
0,1.0,8/13/15,3.732,FEMALE,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,11/2/15,3.510,MALE,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,9/23/15,3.619,MALE,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,7/13/14,4.816,FEMALE,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G","G/, /RR, G/, /RR, G/, /RR, G/, /RR","G/, /RR, G/, /RR, G/, /RR, G/, /RR","/RB, GB/, /RB, GB/, /RB, GB/, /RB, GB/","/RB, GB/, /RB, GB/, /RB, GB/, /RB, GB/"
4,5.0,4/16/15,4.057,FEMALE,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
d = dRaw.loc[ (dRaw['SORT_OUTPUT'].notnull()) | (dRaw['DOUBLE_OUTPUT'].notnull()) | (dRaw['HITCH_OUTPUT'].notnull()) ]

In [4]:
len(d), len(dRaw)

(26, 31)

In [5]:
d = d.loc[ (d['SORT_OUTPUT']!="0") | (d['DOUBLE_OUTPUT']!="0") | (d['HITCH_OUTPUT']!="0") ]

In [6]:
d.tail()

,ID,DOB,AGE,GENDER,SORT_DEMO,SORT_OUTPUT,DOUBLE_DEMO,DOUBLE_OUTPUT,HITCH_DEMO,HITCH_OUTPUT
24,38.0,NaN,NaN,MALE,"G/, /R, G/, /R, G/, /R, G/, /R","R/, /G, R/, /G, R/, /G, R/, /G, R/, /G, R/, /G","R/, /GG, R/, /GG, R/, /GG, R/, /GG","GG/, /R, GG/, /R, GG/, /R, /R, G/, G/, /R, G/","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GB, RB/, /GB, RB/, RBRB/, RBRB..."
27,42.0,NaN,NaN,FEMALE,"G/, /R, G/, /R, G/, /R, G/, /R","R/, /G, R/, /G, RR/, /G, R/, /G, /G, R/, /GG","R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, /G, R/, RR/, /GG, R/, /GG","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GB, RB/, /GB"
28,43.0,NaN,NaN,FEMALE,"G/, /R, G/, /R, G/, /R, G/, /R","/G, R/, /G, R/, /G, /Rc, R/, GGc/, /GG, R/","R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, /GG, R/, /GG, R/, /GG, Rb/, R/, /GG, R/, /GG","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GBBb, /GB, RB/, /GB, RB/, /GB"
29,45.0,NaN,NaN,MALE,"G/, /R, G/, /R, G/, /R, G/, /R","R/, /G, R/, /G, R/, /G, R/, /G, R/, /G","R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, /G, RR/, RRGc/ , /G, R/, R/, R/, /G, /G, /...","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","RB/, /GB, RB/, /GB, RB/, /GB"
30,46.0,NaN,NaN,MALE,"G/, /R, G/, /R, G/, /R, G/, /R",NaN,"R/, /GG, R/, /GG, R/, /GG, R/, /GG","R/, RR/, /Rc, R/, R/, /G, /G","RB/, /GB, RB/, /GB, RB/, /GB, RB/, /GB","B/, G/, /R, /G, RR/"


In [12]:
def regexParse (row, algoType):
    algoLabel = algoType + "_OUTPUT"
    demoLabel = algoType + "_DEMO"
    resString = row[algoLabel]
    dataStr = []
    overall_response_count = 0
    if pandas.isna(resString) : resString = " / "
    for ri, response in enumerate(re.split(r"\s*,\s*", resString)): # split on commas
        lb, rb = re.split(r"/", response) # split left and right buckets

        # remove whitespace
        lb = re.sub(r"\s", "", lb) 
        rb = re.sub(r"\s", "", rb)

        ##############################
        ## Process annotations
        ##############################

        # Remove xes
        lb = re.sub(r"x", "", lb)
        rb = re.sub(r"x", "", rb)

        # b -- put back into original container 
        # for now, we'll remove these
        # sometimes cm may occur if they correct and then put it back
#         lb = re.sub(r"[A-Z][cm]*b", "", lb)
#         rb = re.sub(r"[A-Z][cm]*b", "", rb)
#         or just remove all the b
        lb = re.sub(r"b", "", lb)
        rb = re.sub(r"b", "", rb)

        # add in corrections -- any characters followed by c, put into other bucket
        for x in re.findall("([A-Z])c", lb):
            rb += x
        for x in re.findall("([A-Z])c", rb):
            lb += x
        # now remove these from the original side
        lb = re.sub(r"[A-Z]c", "", lb)
        rb = re.sub(r"[A-Z]c", "", rb)

        # toss out m annotation for now
        lb = re.sub(r"m", "", lb)
        rb = re.sub(r"m", "", rb)

#         print rb + "," + lb
        assert not re.search(r"[^A-Z]", lb)
        assert not re.search(r"[^A-Z]", rb)

        for ai, a in enumerate(lb):
            singleAct = [row["AGE"], algoType, row[demoLabel], row[algoLabel], row['DOB'], row['GENDER'], row['ID'],overall_response_count, ri, ai, "left", a] 
            dataStr.append(singleAct)
            overall_response_count += 1

        for ai, a in enumerate(rb):
            singleAct = [row["AGE"], algoType, row[demoLabel], row[algoLabel], row['DOB'], row['GENDER'], row['ID'],overall_response_count, ri, ai, "right", a] 
            dataStr.append(singleAct)
            overall_response_count += 1
    return dataStr

In [13]:
def parseAll (d):
    dataStr = []
    for subjecti, row in d.iterrows(): 
        dataStr += regexParse(row, "SORT") + regexParse(row, "DOUBLE") + regexParse(row, "HITCH")
    return dataStr
            

In [14]:
d.iloc[0]['SORT_OUTPUT']

'R/, /G, /RGxc, /G, R/, /G'

In [15]:
dataStr = parseAll(d)
print dataStr[0:10]
finalDF = pandas.DataFrame(dataStr, columns=outputCSVHeaders)

[[4.816, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', '7/13/14', 'FEMALE', 4.0, 0, 0, 0, 'left', 'R'], [4.816, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', '7/13/14', 'FEMALE', 4.0, 1, 1, 0, 'right', 'G'], [4.816, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', '7/13/14', 'FEMALE', 4.0, 2, 2, 0, 'left', 'G'], [4.816, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', '7/13/14', 'FEMALE', 4.0, 3, 2, 0, 'right', 'R'], [4.816, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', '7/13/14', 'FEMALE', 4.0, 4, 3, 0, 'right', 'G'], [4.816, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', '7/13/14', 'FEMALE', 4.0, 5, 4, 0, 'left', 'R'], [4.816, 'SORT', 'R/, /G, R/, /G, R/, /G, R/, /G ', 'R/, /G, /RGxc, /G, R/, /G', '7/13/14', 'FEMALE', 4.0, 6, 5, 0, 'right', 'G'], [4.816, 'DOUBLE', 'G/, /RR, G/, /RR, G/, /RR, G/, /RR', 'G/, /RR, G/, /RR, G/, /RR, G/, /RR'

In [16]:
finalDF.head()

,AGE,ALGOTYPE,DEMO,OVERALL.RESPONSE,DOB,GENDER,ID,OVERALL.RESPONSE.NO,RESPONSE.NO,ACTION.NO,BUCKET,RESPONSE
0,4.816,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",7/13/14,FEMALE,4.0,0,0,0,left,R
1,4.816,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",7/13/14,FEMALE,4.0,1,1,0,right,G
2,4.816,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",7/13/14,FEMALE,4.0,2,2,0,left,G
3,4.816,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",7/13/14,FEMALE,4.0,3,2,0,right,R
4,4.816,SORT,"R/, /G, R/, /G, R/, /G, R/, /G","R/, /G, /RGxc, /G, R/, /G",7/13/14,FEMALE,4.0,4,3,0,right,G


In [17]:
finalDF.to_csv("./experimentData.csv")